# Ray13 et al 2013 Evaluation
**Authorship:**
Adam Klie, *08/31/2022*
***
**Description:**
Notebook to perform simple training of models on the Ray et al dataset.
***

In [ ]:
set environment

In [4]:
import kipoi

In [8]:
%set_env GIT_LFS_SKIP_SMUDGE=1

env: GIT_LFS_SKIP_SMUDGE=1


In [9]:
!echo $GIT_LFS_SKIP_SMUDGE

1


In [11]:
!export GIT_LFS_SKIP_SMUDGE=1 && git clone git@github.com:kipoi/models.git

/bin/bash: git: command not found


In [23]:
model = kipoi.get_model('DeepBind/Homo_sapiens/RBP/D00084.001_RNAcompete_A1CF')

Already up to date.
Using downloaded and verified file: /cellar/users/aklie/.kipoi/models/DeepBind/downloaded/model_files/Homo_sapiens/RBP/D00084.001_RNAcompete_A1CF/arch/127f7597630be937c0495a27e6f42e5f
Using downloaded and verified file: /cellar/users/aklie/.kipoi/models/DeepBind/downloaded/model_files/Homo_sapiens/RBP/D00084.001_RNAcompete_A1CF/weights/a4caf255312f44e3fd4e038a79691f2d


2022-08-31 21:41:18.262202: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-31 21:41:31.874842: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-31 21:41:31.878615: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-31 21:41:31.942790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 21:41:31.943942: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:44:00.0 name: Quadro RTX 5000 computeCapability: 7.5
coreClock: 1.815GHz coreCount: 48 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 417.29GiB/s
2022-08-31 21:41:31.943963: I tensorflow/stream_executor/platform/default/d

In [24]:
model

In [25]:
dl_kwargs = model.default_dataloader.download_example('example')

8.19kB [00:00, 27.6kB/s]


303kB [00:00, 1.47MB/s]                            


In [26]:
model.pipeline.predict(dl_kwargs, batch_size=4)

0it [00:00, ?it/s]2022-08-31 21:46:28.055111: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-08-31 21:46:28.069598: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2999865000 Hz
2022-08-31 21:46:28.162324: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-08-31 21:46:30.137635: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
3it [00:22,  7.60s/it]


array([-0.21315557,  0.20937382, -0.29284796,  0.32252452,  0.46879414,
        1.9757667 ,  1.9748571 ,  3.4855967 ,  2.4593873 ,  2.4593873 ],
      dtype=float32)

In [28]:
model.arch

'/cellar/users/aklie/.kipoi/models/DeepBind/downloaded/model_files/Homo_sapiens/RBP/D00084.001_RNAcompete_A1CF/arch/127f7597630be937c0495a27e6f42e5f'

In [29]:
model.dependencies

Dependencies(conda=['h5py=2.10.0', 'tensorflow=2.7.0', 'keras=2.7.0', 'python=3.7', 'bioconda::pysam=0.18.0', 'pip=20.2.4'], pip=[], conda_channels=['defaults'], conda_file=None)

In [31]:
model.predict_on_batch

<bound method KerasModel.predict_on_batch of <kipoi.model.KerasModel object at 0x1555193de450>>

In [14]:
import os

In [16]:
import sys

In [17]:
bin_dir = os.path.dirname(sys.executable)

In [19]:
bin_dir

'/cellar/users/aklie/opt/miniconda3/envs/eugene_benchmarks/bin'

In [20]:
os.environ["PATH"] += os.pathsep + bin_dir

In [21]:
import git

ModuleNotFoundError: No module named 'git'

In [22]:
!git --help

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--super-prefix=<path>] [--config-env=<name>=<envvar>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone     Clone a repository into a new directory
   init      Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add       Add file contents to the index
   mv        Move or rename a file, a directory, or a symlink
   restore   Restore working tree files
   rm        Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect    Use binary search to find th

In [10]:
%%bash
module load git
git --help

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--super-prefix=<path>] [--config-env=<name>=<envvar>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone             Clone a repository into a new directory
   init              Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add               Add file contents to the index
   mv                Move or rename a file, a directory, or a symlink
   restore           Restore working tree files
   rm                Remove files from the working tree and from the index
   sparse-checkout   Initialize and modify the sparse-c